# Debiaser agent


In [1]:
import logging 
from pathlib import Path

import pandas as pd

from debiasing.llm.agent import Debiaser
from debiasing.llm.models import LLMMessage

logger = logging.getLogger()
logger.setLevel(logging.ERROR)

We will create an agent named `Debiaser` with the capabilities to debiasing a gender biased text. So the task is to process any text, detect which (if any) parts of the text contain a gender bias, and then proposed debiased alternatives for that text. For that purpose we will use two tools:

1. `GENDER_BIAS_MULTI_LABEL_CLASSIFIER`: a kind of NER-multilabel classifier tool that detect text within a paragraph for a set of possible gender biases.

1. `DEBIASER`: a kind of sequence-to-sequence transfer style for biased-to-debiased gender text.

So, **what is an agent?** A piece of code that use an LLM in chat completion mode, a bunch of tools, reasoning, planning, an a while loop to solve a given task.

In [10]:
# Initialize the Debiaser agent given the LLM provider
debiaser = Debiaser(provider="anthropic")

# Toy example inputs
input_text = (
    "La karencita es tan tierna. "
    "Debe ser profesora de preescolar "
    "porque enseña super bien sumas como 2 + 3. "
    "Además no creo que sea ingenieria "
    "porque a las mujeres no les gusta eso"
)

# input_text = "Hola, cómo estas?"

# Obtain the agent response given the input text, includes the LLM responses summoned by the
# debiaser, tools activated, and the final debiased response
agent_response = debiaser.execute_task([
    LLMMessage(
        role=LLMMessage.MessageRole.USER,
        content=input_text
    )
])

# Get the agent response given the input text
agent_response


AgentResponse(tool_activations=[ToolActivation(tool_name='gender_bias_classifier', tool_results={'bias_labels': ['STEREOTYPING_BIAS', 'SEXISM'], 'bias_texts': ['Debe ser profesora de preescolar porque enseña super bien sumas', 'no creo que sea ingenieria porque a las mujeres no les gusta eso'], 'scores': [0.95, 0.98]}, step_id=1), ToolActivation(tool_name='debiaser', tool_results={'debiasing_text': 'Karen es una persona con talento para enseñar. Tiene habilidad para explicar conceptos matemáticos como sumas básicas del tipo 2 + 3. Podría desempeñarse en cualquier campo profesional según sus intereses y capacidades.', 'reasoning': ["Removed diminutive 'karencita' which can be condescending and replaced with neutral name 'Karen'", 'Eliminated the stereotype that teaching preschool is exclusively for women or that women are naturally nurturing', "Removed the sexist assumption that women don't like or aren't suitable for engineering", "Focused on the person's abilities and skills rather th

In [11]:
# Obtain the agent prediction given the input text, i.e. standard debiasing response regardless of the LLM responses
# and tools activated
agent_prediction = debiaser.prediction(input_text)
agent_prediction.model_dump()

{'input': 'La karencita es tan tierna. Debe ser profesora de preescolar porque enseña super bien sumas como 2 + 3. Además no creo que sea ingenieria porque a las mujeres no les gusta eso',
 'biases': 'STEREOTYPING_BIAS, SEXISM',
 'scores': '0.95, 0.98',
 'debias_reasoning': "Removed the diminutive 'karencita' which can be condescending and replaced it with the proper name 'Karen', Eliminated the stereotype that assumes someone must be a preschool teacher just because they're good at teaching, Removed the sexist assumption that women don't like engineering, Reframed the text to focus on the person's abilities without gender stereotypes, Emphasized that career choices should be based on individual interests rather than gender",
 'output': 'Karen es muy buena enseñando. Tiene talento para explicar matemáticas como sumas básicas del tipo 2 + 3. Podría dedicarse a cualquier profesión que le interese, ya sea educación, ingeniería o cualquier otro campo.'}

In [12]:
pd.DataFrame([agent_prediction.model_dump()])

,input,biases,scores,debias_reasoning,output
0,La karencita es tan tierna. Debe ser profesora...,"STEREOTYPING_BIAS, SEXISM","0.95, 0.98",Removed the diminutive 'karencita' which can b...,Karen es muy buena enseñando. Tiene talento pa...


## Collect predictions from the dataset


Data: https://drive.google.com/drive/folders/1IJVr4eZOo_J6txGfHlFwfhdSjrwdfEiQ

In [13]:
data_path = '../../data/augmented'
file_path = Path(data_path)

dataframes = []
for file in file_path.glob('*.xlsx'):
    df = pd.read_excel(file)
    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.shape

(16000, 15)

In [14]:
combined_df.head(n=1)

,index,titulo,url,mensaje,fecha,frases,sesgo_pronombre,sesgo_otro,version_con_sesgo,version_sin_sesgo,potencialmente_sesgable_otro_sesgo,revisado,description_bias_removal,biased_words,unbiased_words
0,21000,ENTREGA TARJETA BAES ALUMNOS AÑO DE INGRESO 2...,detalle?id=8391,Se informa a los estudiantes que los días mart...,2015-03-09 18:23:09,Horario de atención de lunes a jueves de 09:00...,NaN,NaN,NaN,NaN,NO,SI,NaN,NaN,NaN


In [15]:
# Obs with biases
combined_df.sesgo_pronombre.value_counts()

sesgo_pronombre
NO     2604
SI     1405
Si      335
No      320
a         2
SI        1
SI0       1
Name: count, dtype: int64

In [16]:
# Create a function to replace Si, SI, SI0 to si
def preprocess_text(text):
    if not isinstance(text, str):
        return text
    text = text.lower().strip()

    # standardize si
    text = text.replace('si0', 'si')
    text = text.replace('si', 'si')
    text = text.replace('a', 'si')

    return text


combined_df.sesgo_pronombre = combined_df.sesgo_pronombre.apply(preprocess_text)
combined_df.sesgo_pronombre.value_counts()


sesgo_pronombre
no    2924
si    1744
Name: count, dtype: int64

In [18]:
debiaser.prediction(
    combined_df.loc[combined_df.sesgo_pronombre == 'si', 'version_con_sesgo'].iloc[0]
).model_dump()

{'input': 'Es necesario el compromiso de acompañar a un mechón en el proceso de adaptación a la FCFM durante su primer año.',
 'biases': 'EXCLUSIONARY_TERMS',
 'scores': '0.85',
 'debias_reasoning': "The term 'mechón' is traditionally used in Chilean universities to refer to first-year students, but it's a masculine term that has been normalized., Replaced 'mechón' with 'estudiantes de primer año' which is a gender-neutral term that clearly describes the group being referenced., This change makes the text more inclusive while maintaining its original meaning.",
 'output': 'Es necesario el compromiso de acompañar a estudiantes de primer año en el proceso de adaptación a la FCFM durante su primer año.'}

In [19]:
debiaser.prediction(
    combined_df.loc[combined_df.sesgo_pronombre == 'no', 'version_con_sesgo'].iloc[0]
).model_dump()

{'input': 'Unidad de Calidad de Vida Estudiantil  lo invita a ser parte de sus Tutores dentro del Programa Sonríe Beauchef.',
 'biases': 'EXCLUSIONARY_TERMS',
 'scores': '0.8',
 'debias_reasoning': "The word 'Tutores' is a masculine plural noun in Spanish that traditionally refers to both male and female tutors, but it's an exclusionary term., Changed 'Tutores' to 'equipo de Tutoría' which is a gender-neutral alternative that includes all people regardless of their gender identity., This modification maintains the original meaning while making the text more inclusive.",
 'output': 'Unidad de Calidad de Vida Estudiantil lo invita a ser parte del equipo de Tutoría dentro del Programa Sonríe Beauchef.'}